In [ ]:
df1


,fit,user_id,bust_size,item_id,weight,rating,rented_for,review_text,body_type,review_summary,category,height,size,age,review_date,sent_score,cluster,fit_type
0,fit,420272,34d,2260466,137.0,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,2016-04-20,2.0,1,small
1,fit,273551,34b,153475,132.0,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,2013-06-18,3.0,2,large
2,fit,360448,34b,1063761,130.0,10.0,party,This hugged in all the right places! It was a ...,hourglass,It was a great time to celebrate the (almost) ...,sheath,64.0,4,116.0,2015-12-14,6.0,2,large
3,fit,909926,34c,126335,135.0,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,2014-02-12,3.0,2,large
4,fit,151944,34b,616682,145.0,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,2016-09-26,3.0,1,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,fit,177016,32b,1475816,125.0,10.0,wedding,"Super comfortable fabric, conservative overall...",petite,Tons of compliments on this dress!,dress,66.0,4,35.0,2017-09-08,2.0,1,small
1022,fit,547738,32d,1340234,128.0,10.0,wedding,Loved this dress and got so many compliments! ...,hourglass,"Easy to wear, unique",dress,68.0,4,30.0,2017-11-06,1.0,2,large
1023,fit,316065,32d,1585757,118.0,10.0,formal affair,I had to wear really high heels so it didn't d...,hourglass,Fancy without being over the top. Got lots of ...,gown,62.0,4,38.0,2017-03-23,4.0,1,small
1024,fit,192770,38dd,126335,130.0,10.0,other,Con: Zipper is a little difficult to work with.,hourglass,Wore this to the after party of my wedding. Go...,dress,67.0,20,27.0,2016-01-22,2.0,1,small


In [ ]:
# results_df = pd.DataFrame({
#     'Metric': [
#         'Mean of actual sizes',
#         'MAE',
#         'MSE',
#         'RMSE',
#         'R2 Score',
#         'MAE%',
#         'RMSE%'
#     ],
#     'Value': [
#         round(mean_y, 2),
#         mae,
#         mse,
#         rmse,
#         r2_score(y_test, preds),
#         round(mae_percent, 2),
#         round(rmse_percent, 2)
#     ]
# })

# ghi ra Excel
#results_df.to_excel("size_model_output.xlsx", index=False)


In [ ]:
#from google.colab import files
#files.download("size_model_output.xlsx")

In [ ]:
# encode categorical variables
df1['body_type_encoded'] = df1['body_type'].astype('category').cat.codes
df1['fit_type_encoded'] = df1['fit_type'].astype('category').cat.codes

# test features
feature_sets = {
    'height, weight, age': ['height', 'weight', 'age'],
    '+ body_type': ['height', 'weight', 'age', 'body_type_encoded'],
    '+ fit_type': ['height', 'weight', 'age', 'fit_type_encoded'],
    '+ body_type + fit_type': ['height', 'weight', 'age', 'body_type_encoded', 'fit_type_encoded']
}

results_split = {}
results_full = {}

#evaluate
def evaluate(X, y):
    model = RandomForestRegressor(n_estimators=500, random_state=42)
    model.fit(X, y)
    pred = model.predict(X)
    return (
        round(mean_absolute_error(y, pred), 3),
        round(np.sqrt(mean_squared_error(y, pred)), 3),
        round(r2_score(y, pred), 3)
    )

# Train-test split 80/20
for label, features in feature_sets.items():
    subset = df1[features + ['size']].dropna()
    X = subset[features]
    y = subset['size']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=500, random_state=42)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    results_split[label] = (
        round(mean_absolute_error(y_test, pred), 3),
        round(np.sqrt(mean_squared_error(y_test, pred)), 3),
        round(r2_score(y_test, pred), 3)
    )

# Train full data
for label, features in feature_sets.items():
    subset = df1[features + ['size']].dropna()
    X = subset[features]
    y = subset['size']
    results_full[label] = evaluate(X, y)

print("=== Train/Test Split (80/20) ===")
for k, v in results_split.items():
    print(f"{k:<30}: MAE = {v[0]}, RMSE = {v[1]}, R2 = {v[2]}")

print("\n=== Full Training (No Split) ===")
for k, v in results_full.items():
    print(f"{k:<30}: MAE = {v[0]}, RMSE = {v[1]}, R2 = {v[2]}")


=== Train/Test Split (80/20) ===
height, weight, age           : MAE = 3.864, RMSE = 5.129, R2 = 0.396
+ body_type                   : MAE = 3.658, RMSE = 5.02, R2 = 0.421
+ fit_type                    : MAE = 3.829, RMSE = 5.111, R2 = 0.4
+ body_type + fit_type        : MAE = 3.634, RMSE = 4.962, R2 = 0.435

=== Full Training (No Split) ===
height, weight, age           : MAE = 1.859, RMSE = 2.737, R2 = 0.853
+ body_type                   : MAE = 1.552, RMSE = 2.208, R2 = 0.904
+ fit_type                    : MAE = 1.664, RMSE = 2.379, R2 = 0.889
+ body_type + fit_type        : MAE = 1.479, RMSE = 2.035, R2 = 0.919


In [ ]:
# Gộp kết quả từ results_split và results_full thành bảng
rows = []
for label in results_split:
    mae_split, rmse_split, r2_split = results_split[label]
    mae_full, rmse_full, r2_full = results_full[label]

    rows.append({
        'Features Used': label,
        'MAE (Split)': mae_split,
        'RMSE (Split)': rmse_split,
        'R2 (Split)': r2_split,
        'MAE (Full)': mae_full,
        'RMSE (Full)': rmse_full,
        'R2 (Full)': r2_full
    })

# Tạo DataFrame
df_eval = pd.DataFrame(rows)

# Xuất ra file Excel
df_eval.to_excel("model_eval_results.xlsx", index=False)


##UI

In [ ]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack


size_model = train_size_predictor(df1)
joblib.dump(size_model, "rec_size_model.pkl")

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_vectorizer.fit(df1['review_summary'].fillna(''))
tfidf_vectors = tfidf_vectorizer.fit_transform(df1['review_summary'].fillna(''))
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")


kmeans_model = KMeans(n_clusters=3, random_state=42)
kmeans_model.fit(tfidf_vectors)
joblib.dump(kmeans_model, "kmeans_model.pkl")


cluster_to_fit = {0: 'fit', 1: 'small', 2: 'large'}
df1['cluster'] = kmeans_model.predict(tfidf_vectors)
df1['fit_type'] = df1['cluster'].map(cluster_to_fit)

lookup_dict = (
    df1.groupby(['body_type', 'fit_type'])['size']
    .median()
    .unstack(fill_value=None)
    .apply(lambda row: row.dropna().to_dict(), axis=1)
    .to_dict()
)
joblib.dump(lookup_dict, "fallback_lookup_dict.pkl")


numeric_cols = ['weight', 'height', 'age', 'rating', 'sent_score']
df_numeric = df1[df1['sent_score'] > 0][numeric_cols].fillna(df1[numeric_cols].mean())
scaler = StandardScaler()
numeric_vectors = scaler.fit_transform(df_numeric)

combined_vectors = hstack([
    tfidf_vectorizer.transform(df1[df1['sent_score'] > 0]['review_summary'].fillna('')),
    numeric_vectors
]).tocsr()

joblib.dump(combined_vectors, "tfidf_knn_vectors.pkl")


knn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
knn_model.fit(combined_vectors)
joblib.dump(knn_model, "knn_model.pkl")


In [ ]:
joblib.dump(size_model, "rec_size_model.pkl")
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(kmeans_model, "kmeans_model.pkl")
joblib.dump(knn_model, "knn_model.pkl")
joblib.dump(combined_vectors, "tfidf_knn_vectors.pkl")
joblib.dump(lookup_dict, "fallback_lookup_dict.pkl")

In [ ]:
import zipfile
from google.colab import files

with zipfile.ZipFile("model_files.zip", "w") as zipf:
    zipf.write("rec_size_model.pkl")
    zipf.write("tfidf_vectorizer.pkl")
    zipf.write("kmeans_model.pkl")
    zipf.write("knn_model.pkl")
    zipf.write("tfidf_knn_vectors.pkl")
    zipf.write("fallback_lookup_dict.pkl")

files.download("model_files.zip")